# Parameter Tunning using GridSearchCV and saving our Model for Deployment Purpose

In [3]:
from sklearn.svm import SVC
from sklearn import datasets
iris=datasets.load_iris()

In [19]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(iris.data,iris.target,test_size=0.3)

In [20]:
model=SVC(kernel='rbf',gamma='auto',C=30)
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.9555555555555556

In [21]:
#KFold Cross Validation
from sklearn.model_selection import cross_val_score
cross_val_score(SVC(kernel='rbf',gamma='auto',C=30),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.9       , 0.93333333, 1.        ])

In [22]:
cross_val_score(SVC(kernel='linear',gamma='auto',C=30),iris.data,iris.target,cv=5)

array([1. , 1. , 0.9, 0.9, 1. ])

In [23]:
cross_val_score(SVC(kernel='rbf',gamma='auto',C=10),iris.data,iris.target,cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [24]:
cross_val_score(SVC(kernel='linear',gamma='auto',C=10),iris.data,iris.target,cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [26]:
import numpy as np
kernels=['rbf','linear']
C=[1,10,20,25,30]
avg_score={}
for kval in kernels:
    for cval in C:
        cv_score=cross_val_score(SVC(kernel=kval,C=cval,gamma='auto'),iris.data,iris.target,cv=5)
        avg_score[kval+" -- "+str(cval)]=np.average(cv_score)

            

In [27]:
avg_score

{'rbf -- 1': 0.9800000000000001,
 'rbf -- 10': 0.9800000000000001,
 'rbf -- 20': 0.9666666666666668,
 'linear -- 1': 0.9800000000000001,
 'linear -- 10': 0.9733333333333334,
 'linear -- 20': 0.9666666666666666}

In [29]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(SVC(gamma='auto'),
                {
                    'C':[1,10,20],
                    'kernel':['rbf','linear']                    
                },cv=5, return_train_score=False)
clf.fit(iris.data,iris.target)
clf.cv_results_

{'mean_fit_time': array([0.00080018, 0.0006    , 0.00080009, 0.00079999, 0.00099998,
        0.00040007]),
 'std_fit_time': array([4.00090228e-04, 4.89901382e-04, 4.00042545e-04, 3.99994861e-04,
        9.53674316e-08, 4.89979265e-04]),
 'mean_score_time': array([0.00020003, 0.00040002, 0.00040002, 0.00020003, 0.00020003,
        0.0006001 ]),
 'std_score_time': array([0.00040007, 0.00048992, 0.00048992, 0.00040007, 0.00040007,
        0.00048998]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20

In [30]:
import pandas as pd
df=pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.0008,4.000902e-04,0.0002,0.00040,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.0006,4.899014e-04,0.0004,0.00049,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.0008,4.000425e-04,0.0004,0.00049,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.0008,3.999949e-04,0.0002,0.00040,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.0010,9.536743e-08,0.0002,0.00040,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.0004,4.899793e-04,0.0006,0.00049,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [32]:
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [34]:
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [36]:
clf.best_score_

0.9800000000000001

In [38]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'refit_time_',
 'return_train_score',
 'score

In [41]:
from sklearn.model_selection import RandomizedSearchCV
rs=RandomizedSearchCV(SVC(gamma='auto'),
                     {
                        'C':[1,10,20],
                        'kernel':['rbf','linear']                    
                    },cv=5, return_train_score=False, n_iter=2)
rs.fit(iris.data,iris.target)
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
                     

,param_C,param_kernel,mean_test_score
0,1,linear,0.980000
1,10,linear,0.973333


In [42]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}



In [45]:
scores=[]

In [46]:
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.966667,{'n_estimators': 5}
2,logistic_regression,0.966667,{'C': 5}


In [ ]:
#from above results we can conclude that SVM Works best with C=1, Kernel RBF